# System Set up

In [ ]:
!pip install nltk
!pip install sklearn
!pip install pandas
!pip install numpy
!pip install re 
!pip install lyricsgenius
!pip install keras
!pip install tensorflow


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
     |████████████████████████████████| 59 kB 4.6 MB/s 


In [ ]:
import lyricsgenius as lg
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from collections import Counter
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Activation, Dropout
import numpy as np 
#from tensorflow import keras
#from __future__ import print_function

#from tensorflow.keras.optimizers import Adam
#from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

# Getting the lyrics from Genius API


If the API times out before you are able to get 200 songs, skip to the section titled "Testing and Training Split", and get the data from Github.

In [ ]:
token = 'AfdLFfPg5oAQFc-mF6EcotZXRkH_TFBRE9eDdbQ5RkZwtBuVGXJyBlfDrkiCByBt'
artists = []
genius = lg.Genius(token, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)","Speech","Tour","Poem","(Voice Memo)", "Magazine", "A Message From Taylor"],remove_section_headers = True) # token

while True:
    try:
        artists.append(genius.search_artist("Taylor Swift", max_songs=200,sort="popularity"))
        break
    except:
        pass

Searching for songs by Taylor Swift...

Song 1: "​cardigan"
Song 2: "All Too Well (10 Minute Version) (Taylor’s Version) [From the Vault]"
Song 3: "​exile"
Song 4: "Lover"
Song 5: "​the 1"
Song 6: "Look What You Made Me Do"
Song 7: "​betty"
Song 8: "​august"
Song 9: "End Game"
Song 10: "You Need To Calm Down"
Song 11: "Blank Space"
Song 12: "​champagne problems"
Song 13: "ME!"
Song 14: "...Ready for It?"
Song 15: "​my tears ricochet"
Song 16: "​willow"
Song 17: "​invisible string"
Song 18: "Cruel Summer"
Song 19: "​the last great american dynasty"
Song 20: "All Too Well"
Song 21: "​seven"
Song 22: "Delicate"
Song 23: "Call It What You Want"
Song 24: "Style"
Song 25: "​illicit affairs"
Song 26: "​this is me trying"
"Bad Blood (Remix)" is not valid. Skipping.
"I Don’t Wanna Live Forever" is not valid. Skipping.
Song 27: "Love Story"
Song 28: "​evermore"
Song 29: "​happiness"
Song 30: "​peace"
Song 31: "Gorgeous"
"All Too Well (10 Minute Version) (Taylor’s Version) [Live Acoustic]" is not

In [41]:
cList = {"ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because","could've": "could have",
"couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not",
"hadn't've": "had not have","hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have",
"he's": "he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have",
"I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","isn't": "is not","it'd": "it had","it'd've": "it would have",
"it'll": "it will","it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
"might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not",
"mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have","she'd": "she would",
"she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is","should've": "should have",
"shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would",
"that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are",
"they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had","we'd've": "we would have","we'll": "we will",
"we'll've": "we will have","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will","what'll've": "what will have",
"what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have","where'd": "where did",
"where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have",
"why's": "why is","why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have",
"wouldn't": "would not","wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
"y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have",
"you'll": "you you will","you'll've": "you you will have","you're": "you are","you've": "you have", "'cause":'because'}
c_re = re.compile('(%s)' % '|'.join(cList.keys())) #regular expression object, with format of contractions like i'm, i've
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)] 
    return c_re.sub(replace, text)  

#### removing stopwords and non-lyric stuff
songs = artists[0].songs
lyrics_wo_stopword=[]
for i in songs:
    lyric = expandContractions(i.lyrics.lower()) # expanding abbreviation to two words format
    lyric  = re.sub(r"\W+"," ",lyric) 
    lyric  = re.sub(r"[a-z]*\d+embedshare"," ",lyric)
    lyric  = re.sub(r"urlcopyembedcopy"," ",lyric)
    lyric_token = word_tokenize(lyric)
    filtered_words = [word.lower() for word in lyric_token  if word.lower() not in stopwords.words('english')]
    lyrics_wo_stopword.append(' '.join(filtered_words))

# Assigning Sentiment Tags to Lyrics


In [42]:
sia = SentimentIntensityAnalyzer()
lyric_df = pd.DataFrame(lyrics_wo_stopword,columns =['lyrics'])
lyric_df['pos']  = lyric_df['lyrics'].apply(lambda x: sia.polarity_scores(x)['pos'])
lyric_df['neg']  = lyric_df['lyrics'].apply(lambda x: sia.polarity_scores(x)['neg'])
lyric_df['compound']  = lyric_df['lyrics'].apply(lambda x: sia.polarity_scores(x)['compound'])
lyric_df['sentiment'] = lyric_df['compound'].apply(lambda x: 'pos' if (x>0) else 'neg') #using compound score, if >0 then assign it as positive text

In [43]:
lyric_df.sentiment.value_counts() # for now we have 101 positive tagged lyrics 

pos    153
neg     47
Name: sentiment, dtype: int64

In [44]:
# sanity check if we exclude all the none lyrics stuff 
# by looking at the length of the lyrics 
total_word_count = lyric_df['lyrics'].str.split().apply(len)
for index, value in total_word_count.items():
    print(str(index) +':' + str(value) )

0:160
1:388
2:219
3:110
4:143
5:217
6:177
7:172
8:368
9:187
10:264
11:139
12:248
13:198
14:121
15:190
16:160
17:224
18:170
19:189
20:97
21:161
22:245
23:201
24:121
25:93
26:168
27:144
28:167
29:135
30:180
31:133
32:115
33:153
34:172
35:139
36:222
37:257
38:298
39:133
40:188
41:273
42:107
43:75
44:176
45:132
46:225
47:226
48:210
49:263
50:180
51:232
52:239
53:109
54:205
55:176
56:153
57:123
58:164
59:260
60:169
61:157
62:150
63:152
64:146
65:161
66:218
67:211
68:184
69:119
70:151
71:149
72:198
73:107
74:157
75:247
76:314
77:198
78:94
79:130
80:184
81:242
82:174
83:216
84:145
85:254
86:177
87:172
88:123
89:123
90:184
91:255
92:193
93:212
94:218
95:193
96:188
97:132
98:80
99:164
100:168
101:156
102:118
103:170
104:192
105:262
106:187
107:179
108:148
109:236
110:220
111:128
112:205
113:151
114:185
115:158
116:188
117:118
118:100
119:180
120:193
121:138
122:101
123:171
124:113
125:133
126:147
127:81
128:199
129:116
130:150
131:148
132:183
133:119
134:136
135:155
136:176
137:146
138:165
139:

# Testing and Training Split

In [ ]:
# uncomment line below if API keeps timing out (fetched from the results using the same script)
# lyric_df = pd.read_csv('Taylor_200_song_lyrics_with_sentiment.csv', header = 0)

In [ ]:
train, test = train_test_split(lyric_df, test_size=0.2, random_state=123)

In [ ]:
print(train.shape)
print(test.shape)

(160, 5)
(40, 5)


# Naive Bayes Model


In [ ]:
def get_text(df,  tag):
  # Join together the text in the reviews for a particular class.
    t = df.loc[df['sentiment'] == tag, ['lyrics']]
    return ' '.join(t.lyrics)

def count_text(text):
  # Count up the occurence of each word.
    lyric_token = word_tokenize(text)
    return Counter(lyric_token)

In [ ]:
train_text_neg = get_text(train, 'neg')
neg_word_cnt = count_text(train_text_neg)
train_text_pos = get_text(train, 'pos')
pos_word_cnt = count_text(train_text_pos)
voc_word_cnt = neg_word_cnt + pos_word_cnt 
len(set(voc_word_cnt))

3195

In [ ]:
def train_naive_bayes( tag, word_cnt, voc_word_cnt):
    """
    Get the log likelihood of a word existing in a certain class
    Use log to avoid underflow
    Input: 
        1.tag: the sentiment tag, 'pos' or 'neg'
        2.word_cnt: the word count frequency for each class
        3.voc_word_cnt: the word count frequency for all the training documents
    Output: 
        1.prior: the probability of getting a class from the document 
        2.prob_dict: the log likelihood of seeing this word in a particular class
    """
    prior = np.log(len(train.loc[train['sentiment'] == tag,]) / len(train))
    prob_dict = dict()
    for word in voc_word_cnt: 
        prob = np.log((word_cnt.get(word,0) +1 ) / (sum(word_cnt.values())+ len(voc_word_cnt))) 
        prob_dict[word] = prob
    return prior, prob_dict

In [ ]:
def make_class_prediction(text, voc,  prior, prob):
    """
    Given the trained data, predict whether the new text is positive or negative 
    Input:
        1. text: new text that we want to predict
        2. voc: the vocabulary list from training data set. If the word in text is not in the training set, ignore it
        3. prior: the probability of getting a class from the document using the training data 
        4. prob: the probability of a word appearing in a specific class using the training data
    Output:
        1. prediction: the probability of the text being in a specific class
    """
    prediction = prior
    text_counts = Counter(count_text(text))
    for word in text_counts:
        if word in voc.keys():
            p = prob[word]
            prediction +=  np.log(text_counts.get(word)) + p
    return prediction 

In [ ]:
testing_bayes = test
# test it with the first testing set 
testing_bayes['pos_pred'] = testing_bayes['lyrics'].apply(lambda x: make_class_prediction(x, voc_word_cnt 
                    , train_naive_bayes('pos',pos_word_cnt, voc_word_cnt)[0]
                    , train_naive_bayes('pos',pos_word_cnt, voc_word_cnt)[1]))

testing_bayes['neg_pred'] = testing_bayes['lyrics'].apply(lambda x: make_class_prediction( x, voc_word_cnt
                    , train_naive_bayes('neg',neg_word_cnt, voc_word_cnt)[0]
                    , train_naive_bayes('neg',neg_word_cnt, voc_word_cnt)[1]))

In [ ]:
# assign predicted label by taking argmax of the two predictions
testing_bayes['pred_label'] = np.where(testing_bayes['pos_pred']>testing_bayes['neg_pred'], 
                                           'pos', 'neg')

In [ ]:
# testing accuracy
sum(testing_bayes['pred_label']==testing_bayes['sentiment'])/len(testing_bayes)

0.875

In [ ]:
training_bayes = train
# train it with the first training set 
training_bayes['pos_pred'] = training_bayes['lyrics'].apply(lambda x: make_class_prediction( x , voc_word_cnt
                    , train_naive_bayes('pos',pos_word_cnt, voc_word_cnt)[0]
                    , train_naive_bayes('pos',pos_word_cnt, voc_word_cnt)[1]))

training_bayes['neg_pred'] = training_bayes['lyrics'].apply(lambda x: make_class_prediction( x, voc_word_cnt
                    , train_naive_bayes('neg',neg_word_cnt, voc_word_cnt)[0]
                    , train_naive_bayes('neg',neg_word_cnt, voc_word_cnt)[1]))

In [ ]:
# assign predicted label by taking argmax of the two predictions
training_bayes['pred_label'] = np.where(training_bayes['pos_pred']>training_bayes['neg_pred'], 
                                           'pos', 'neg')

In [ ]:
# training accuracy
sum(training_bayes['pred_label']==training_bayes['sentiment'])/len(training_bayes)

1.0

## Generate Synthetic Data



In this part, we want to generate synthetic data from the trained probabilities in Naive Bayes Section



In [ ]:
# normalizing the probability to ensure they sum up to 1 
def normalize(probs):
    prob_factor = 1 / sum(probs)
    return [prob_factor * p for p in probs]

In [ ]:
total_word_count = lyric_df['lyrics'].str.split().apply(len)
mu = total_word_count.mean() # get the mean of lyrics length 
sigma = np.std(total_word_count) # get the standard deviation of lyrics length 
print(mu, sigma)

169.77 51.378663859621724


In [ ]:
def generate_synthetic_data(tag, n, mu, sigma, word_prob):
    """
    Generating the synthtic data using the distribution of the true lyrics set 
    Input: 
        1. tag: positive class of negative class 
        2. n : number of documents you want to generate for each tag
        3. mu: the mean of distribution of the true lyrics set 
        4. sigma: the stdev of distribution of the 
        5. word_prob: the probability of each word in a certain class from the training dataset 
    Output: 
        1. a dictionary with tag and the list of sample words
    """
    syn_data = []
    document_length_sampler = [round(num, 0) for num in np.random.normal(mu, sigma, n).tolist()]
    for i in range(n): 
        sample_length= int(document_length_sampler[i])
        # sample with replacement 
        sample_pos_doc = np.random.choice(list(word_prob.keys()), sample_length
                                      , p=normalize(list(word_prob.values()))).tolist()
        syn_data.append(' '.join(sample_pos_doc))
    return {'lyrics':syn_data, 'sentiment':[tag]*n}

In [ ]:
# exponentiate each probability we trained for naive bayes cuz we use log to avoid underflow 
# each word probability dictionary itself should already sum up to 1 
pos_word_prob ={k: np.exp(v) for k, v in train_naive_bayes('pos',pos_word_cnt, voc_word_cnt)[1].items()}
neg_word_prob ={k: np.exp(v) for k, v in train_naive_bayes('neg',neg_word_cnt, voc_word_cnt)[1].items()}

In [ ]:
pos_syn= generate_synthetic_data('pos', 200, mu, sigma, pos_word_prob);
neg_syn= generate_synthetic_data('neg', 200, mu, sigma, neg_word_prob)

In [ ]:
syn_data  = pd.concat([pd.DataFrame(pos_syn), pd.DataFrame(neg_syn)])
syn_data = shuffle(syn_data) # shuffle the order to mix up positive data and negative dta 

In [ ]:
# test it with the first testing set 
syn_data['pos_pred'] = syn_data['lyrics'].apply(lambda x: make_class_prediction(x, voc_word_cnt 
                    , train_naive_bayes('pos',pos_word_cnt, voc_word_cnt)[0]
                    , train_naive_bayes('pos',pos_word_cnt, voc_word_cnt)[1]))

syn_data['neg_pred'] = syn_data['lyrics'].apply(lambda x: make_class_prediction( x, voc_word_cnt
                    , train_naive_bayes('neg',neg_word_cnt, voc_word_cnt)[0]
                    , train_naive_bayes('neg',neg_word_cnt, voc_word_cnt)[1]))

In [ ]:
# assign predicted label by taking argmax of the two predictions
syn_data['pred_label'] = np.where(syn_data['pos_pred']>syn_data['neg_pred'], 
                                           'pos', 'neg')

In [ ]:
# testing accuracy for synthetic data 
sum(syn_data['pred_label']==syn_data['sentiment'])/len(syn_data)

1.0

#LSTM Model


## Training LSTM Model

In [ ]:
total_word_count = lyric_df['lyrics'].str.split().apply(len)
max_features = max(total_word_count)
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(lyric_df['lyrics'].values)
X = tokenizer.texts_to_sequences(lyric_df['lyrics'].values)
X = pad_sequences(X)

In [ ]:
X

array([[  0,   0,   0, ..., 361, 118,  23],
       [  0,   0,   0, ..., 363, 226,  42],
       [  0,   0,   0, ...,   4, 195,  27],
       ...,
       [  0,   0,   0, ...,  29, 290, 121],
       [  0,   0,   0, ..., 230,  68,   5],
       [  0,   0,   0, ..., 127,  46,  85]], dtype=int32)

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=X.shape[1]))
model.add(LSTM(16))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 277, 64)           24832     
                                                                 
 lstm (LSTM)                 (None, 16)                5184      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 30,033
Trainable params: 30,033
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
lyric_df['sentiment_bin'] = lyric_df['compound'].apply(lambda x: 1 if (x>0) else 0) #using compound score, if >0 then assign it as positive text
Y = lyric_df['sentiment_bin'].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state=123)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(160, 277) (160,)
(40, 277) (40,)


In [ ]:
batch_size = 10
model.fit(X_train,
          Y_train,
          epochs=6, batch_size = batch_size,
          verbose = 'auto'
           )

Epoch 1/6
16/16 [==============================] - 3s 61ms/step - loss: 0.6761 - accuracy: 0.7000
Epoch 2/6
16/16 [==============================] - 1s 63ms/step - loss: 0.6037 - accuracy: 0.7375
Epoch 3/6
16/16 [==============================] - 1s 64ms/step - loss: 0.4831 - accuracy: 0.7375
Epoch 4/6
16/16 [==============================] - 1s 62ms/step - loss: 0.3923 - accuracy: 0.7812
Epoch 5/6
16/16 [==============================] - 1s 63ms/step - loss: 0.2634 - accuracy: 0.8750
Epoch 6/6
16/16 [==============================] - 1s 64ms/step - loss: 0.1753 - accuracy: 0.9438


### Predicting LSTM model with Real Data (Training and Testing Accuracy)

In [ ]:
predictions_nn_train = model.predict(X_train)
predictions_nn_test = model.predict(X_test)
for i in range(len(predictions_nn_train)):
    if predictions_nn_train[i][0] < 0.5:
        predictions_nn_train[i][0] = 0
    else:
        predictions_nn_train[i][0] = 1
        
for i in range(len(predictions_nn_test)):
    if predictions_nn_test[i][0] < 0.5:
        predictions_nn_test[i][0] = 0
    else:
        predictions_nn_test[i][0] = 1

In [ ]:
print('Train accuracy:', accuracy_score(Y_train, predictions_nn_train))
print('Test accuracy', accuracy_score(Y_test, predictions_nn_test))

Train accuracy: 0.9875
Test accuracy 0.875


### Predicting LSTM model with Synthetic Data

In [ ]:
total_word_count_syn = syn_data['lyrics'].str.split().apply(len)
max_features_syn = max(total_word_count_syn)
max_features_syn

317

In [ ]:
tokenizer_syn = Tokenizer(num_words=max_features_syn, split=' ')
tokenizer_syn.fit_on_texts(syn_data['lyrics'].values)
X_syn = tokenizer.texts_to_sequences(syn_data['lyrics'].values)
X_syn = pad_sequences(X_syn)

In [ ]:
syn_data['sentiment_bin'] = syn_data['sentiment'].apply(lambda x: 1 if (x =='pos') else 0) 
Y_syn = syn_data['sentiment_bin'].values

In [ ]:
#### LSTM with synthetic 
predictions_syn = model.predict(X_syn)
for i in range(len(predictions_syn)):
    if predictions_syn[i][0] < 0.5:
        predictions_syn[i][0] = 0
    else:
        predictions_syn[i][0] = 1


In [ ]:
print('Synthetic data accuracy:', accuracy_score(Y_syn, predictions_syn))


Synethetic data accuracy: 0.77
